# Our First PySpark Notebook on Databricks

In this first notebook we will cover some of the basic functionalities of PySpark. In particular, we will discuss how to play with data using PySpark's <code>DataFrame</code> as building block.
The first part of the notebook will show how <code>DataFrame</code> objects can be created "manually", whilst in the second part we will see how to load _external_ data sources into a <code>DataFrame</code> object. Of course, the latter is way more useful as most of the times we will be needing to work with (possibly many) external data sources.

We will start our journey just checking that our PySpark environment hosted on Databricks platform is up and running properly!

## Import useful PySpark packages

In [0]:
import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

## Check everything is ok

In [0]:
spark

In [0]:
sc._conf.getAll()

# **Create PySpark's <code>DataFrame</code> "manually"**

Let's first create <code>Employee</code> and <code>Department</code> <code>Row</code> instances.

In [0]:
# Row is a generic "record" object with an ordered collection of fields that can be accessed by index or name
# In this case, we use Row to create the schema of other Row objects
Employee = Row("firstName", "lastName", "email", "salary")

# Create some Row instances following the above schema
employee_1 = Employee('Basher', 'armbrust', 'bash@edureka.co', 100000)
employee_2 = Employee('Daniel', 'meng', 'daniel@stanford.edu', 120000 )
employee_3 = Employee('Muriel', None, 'muriel@waterloo.edu', 140000 )
employee_4 = Employee('Rachel', 'wendell', 'rach_3@edureka.co', 160000 )
employee_5 = Employee('Zach', 'galifianakis', 'zach_g@edureka.co', 160000 )

# Print out a whole Row instance
print(employee_3)

In [0]:
# Access and print out the first Row field of `employee_1` by index
print("Employee 1's first name is: {:s}".format(employee_1[0]))

# Access and print out the `salary` Row field of `employee_4` by name
print("Employee 4's salary is: {:d}".format(employee_4.salary))

In [0]:
# Create another set of Row objects. 
# This time, schema is defined by means of 2 named attributes, i.e., `id` and `name`.
# Each Row object specifies those attribute at creation time 
# (rather than having a `Department` Row object containing only the schema, as we did before with `Employee`)
department_1 = Row(id='123456', name='HR')
department_2 = Row(id='789012', name='OPS')
department_3 = Row(id='345678', name='FN')
department_4 = Row(id='901234', name='DEV')

Now, we'll create a <code>DepartmentWithEmployees</code> <code>Row</code> instance from the Employee and Departments.

In [0]:
departmentWithEmployees_1 = Row(department=department_1, employees=[employee_1, employee_2, employee_5])
departmentWithEmployees_2 = Row(department=department_2, employees=[employee_3, employee_4])
departmentWithEmployees_3 = Row(department=department_3, employees=[employee_1, employee_4, employee_3])
departmentWithEmployees_4 = Row(department=department_4, employees=[employee_2, employee_3])

In [0]:
print(departmentWithEmployees_2)

Let's create our <code>DataFrame</code> from the list of rows above

In [0]:
departmentsWithEmployees = [departmentWithEmployees_1, 
                            departmentWithEmployees_2, 
                            departmentWithEmployees_3,                           
                            departmentWithEmployees_4]

# Create the Spark's DataFrame object
df = spark.createDataFrame(departmentsWithEmployees)
# Print out the DataFrame
df.show(n=2, truncate=False) # try, playing with the arguments (e.g., truncate=True/False)

# **Create PySpark's <code>DataFrame</code> from an external input source**

Most of the times, data we will be working with are not manually created as above, yet they are located at several remote repositories and herein stored in various formats like <code>csv</code>, <code>json</code>, <code>xml</code>, or [Parquet](https://spark.apache.org/docs/latest/sql-data-sources-parquet.html) files. Moreover, data can be loaded either from _local_ or _distributed_ file systems (e.g., HDFS or Amazon S3).

Remember, though, that our development environment is provided by Databricks (Community Edition) which - differently from classical Jupyter Notebook that usually is executed on our local machine - runs on the Databricks' cloud infrastructure. As such, "_local_" file system **in this case** means local to Databricks' cloud! In other words, any references to data made within this notebook is relative to the Databrick's end side. That is why we need Databricks to know how to get our data before we can make use of it.

Databricks makes use of its own distributed file system, which is called [Databricks File System (DBFS)](https://docs.databricks.com/data/databricks-file-system.html). As such, we must load our data to DBFS **before** we can actually use it!

## **1. Data Acquisition (from a remote source)**

If you have a dataset stored on your local machine and you want to make it available to DBFS you can easily take advantage of the **Data** menu item on the left-hand panel. Before you can access that menu, though, you must be sure the option _DBFS File Browser_ in the _Advanced Settings_ of the _Admin Console_ is **enabled** (by default, it is not). Once this is done, you can simply refer to this [link](https://docs.databricks.com/data/databricks-file-system.html#file-upload-interface) for any further instructions.

However, most of the time, datasets are not stored on your local machine, yet they are located in remote repositories (i.e., external sources). As such, if you want to use the DBFS File Browser you may need to proceed as follows:
1. Download the dataset from its original remote location to your own local file system;
2. Upload the dataset from your own local file system to DBFS.
 
To avoid the 2-step procedure above - which may cause several latency due to multiple network transfers - you would rather prefer to download/store a remote dataset file to DBFS **directly**.
Let's see how!

#### DBFS vs. Local Driver Node's File System

You can work with files on DBFS or on the local driver node of the cluster. Either way, there are **4 main options** to access files:
1. ```%fs```
2. ```%sh```
3. Databricks file system utilities (```dbutils.fs```)
4. Single-node file system APIs (```os```)

The first two are **magic commands** integrated with Notebook.

##### Accessing files on DBFS
The path to the default blog storage (**root**) is ```dbfs:/```. 
The default location for ```%fs``` and ```dbutils.fs``` is **root**. Thus, to read from or write to root you can either use:

```%fs command /path/```<br>
or<br>
```dbutils.fs command ("/<path>/")```

Both ```%sh``` and single-node file system APIs read from the local files ystem by default (i.e., the local file system of the driver node). To access root or mounted paths in root with ```%sh``` or file system APIs, you need to preface the path with ```/dbfs/```:

```%sh <command> /dbfs/<path>/```<br>
or<br>
```
import os 
os.<command>("/dbfs/<path>/")
```

##### Accessing files on the local driver node's file system
```%fs``` and ```dbutils.fs``` read by default from **root** (```dbfs:/```). To read from the local file system, you must prepend ```file:/```

```%fs <command> file:/<path>/```<br>
or<br>
```dbutils.fs.<command> ("file:/<path>/")```

Both ```%sh``` and single-node file system APIs read from the local files ystem by default, therefore:

```%sh <command> /<path>/```<br>
or<br>
```
import os 
os.<command>("/<path>/")
```
<hr>

##### To Wrap Up
<center>
<table border="1" class="docutils">
<colgroup>
<col width="18%" />
<col width="22%" />
<col width="24%" />
<col width="37%" />
</colgroup>
<thead valign="bottom">
<tr class="row-odd"><th class="head">Command</th>
<th class="head">Default location</th>
<th class="head">To read from root</th>
<th class="head">To read from local filesystem</th>
</tr>
</thead>
<tbody valign="top">
<tr class="row-even"><td> <code class="docutils literal notranslate"><span class="pre">%fs</span></code></td>
<td> Root</td>
<td>&#160;</td>
<td> Add <code class="docutils literal notranslate"><span class="pre">file:/</span></code> to path</td>
</tr>
<tr class="row-odd"><td> <code class="docutils literal notranslate"><span class="pre">%sh</span></code></td>
<td> Local driver node</td>
<td> Add <code class="docutils literal notranslate"><span class="pre">/dbfs</span></code> to path</td>
<td>&#160;</td>
</tr>
<tr class="row-even"><td>  <code class="docutils literal notranslate"><span class="pre">dbutils.fs</span></code></td>
<td> Root</td>
<td>&#160;</td>
<td> Add <code class="docutils literal notranslate"><span class="pre">file:/</span></code> to path</td>
</tr>
<tr class="row-odd"><td>  <code class="docutils literal notranslate"><span class="pre">os.&lt;command&gt;</span></code></td>
<td> Local driver node</td>
<td> Add <code class="docutils literal notranslate"><span class="pre">/dbfs</span></code> to path</td>
<td>&#160;</td>
</tr>
</tbody>
</table>
</center>

<center>![Accessing Files](https://docs.databricks.com/_images/dbfs-and-local-file-paths.png)</center>

#### List all directories from DBFS root (i.e., ```dbfs:/```)

In [0]:
%fs ls

In [0]:
dbutils.fs.ls("/")

In [0]:
%sh ls /dbfs/
# NOTE: Starting from Databricks Runtime 7 this mount point is disabled (https://stackoverflow.com/a/63814218)

In [0]:
import os
os.listdir("/dbfs/")

#### List all directories from local driver node's root

In [0]:
%fs ls file:/

In [0]:
dbutils.fs.ls("file:/")

In [0]:
%sh ls /

In [0]:
import os
os.listdir("/")

### 1.1 Create a dedicated <code>bdc-2020-21</code> folder on DBFS
As first step, let's create a dedicated folder on DBFS and double-check this leads to what we expect.

**NOTE:** This is a one-time step; once you do it you will have access to the DBFS folder also in the future.

In [0]:
%fs mkdirs bdc-2020-21/datasets/

In [0]:
%fs ls

In [0]:
%fs ls /bdc-2020-21/

### 1.2 Download the dataset to the local driver node's ```/tmp``` folder using ```wget```

In [0]:
%sh wget -P /tmp https://github.com/gtolomei/big-data-computing/raw/master/datasets/fifa-players-2020.csv.bz2

In [0]:
%fs ls file:/tmp/

### 1.3 Move the file from local driver node's file system to DBFS

In [0]:
dbutils.fs.mv("file:/tmp/fifa-players-2020.csv.bz2", "dbfs:/bdc-2020-21/datasets/fifa-players-2020.csv.bz2")

In [0]:
%fs ls file:/tmp/

In [0]:
%fs ls /bdc-2020-21/datasets/

### 1.4 Load dataset from DBFS to Spark ```DataFrame```

In [0]:
fifa_df = spark.read.load("dbfs:/bdc-2020-21/datasets/fifa-players-2020.csv.bz2", 
                           format="csv", 
                           sep=",", 
                           inferSchema="true", 
                           header="true")

### **1.5 Display the first <code>n=5</code> rows of the loaded dataset**

In [0]:
fifa_df.show(n=5, truncate=False)

### **1.6 Print out the schema of the DataFrame**

In [0]:
fifa_df.printSchema()

### **1.7 Check the shape of the loaded dataset, i.e., number of rows and columns**

In [0]:
print("The shape of the dataset is {:d} rows by {:d} columns".format(fifa_df.count(), len(fifa_df.columns)))

### **1.8 Check the data types of the loaded dataset**

In [0]:
print(fifa_df.dtypes)

### **1.9 Describing a particular column**

If we want to have a look at the summary of any particular column of a DataFrame, we use the <code>describe</code> method. This method gives us the statistical summary of the given column if not specified, it provides the statistical summary of the DataFrame.

In [0]:
# Statistical summary of the `age` column
fifa_df.describe("age").show()

In [0]:
# Statistical summary of the `height_cm` column
fifa_df.describe("height_cm").show()

## **2. Data Manipulation**

Once data has been successfully loaded into a Spark DataFrame, we can start working with it. Most of the operations involve: _selecting_, _filtering_, _sorting_, _grouping_, and compute _aggregate functions_ (e.g., _count_).

### **2.1 Selecting Multiple Columns**

If we want to select particular columns from the DataFrame, we use the <code>select</code> method.

In [0]:
fifa_df.select(["long_name", "club", "nationality"]).show(n=10, truncate=False)
# Alternatively (no list notation):
# fifa_df.select("long_name", "club", "nationality").show(n=10, truncate=False)

If we want to select (multiple) **distinct** columns, we will use the <code>distinct</code> method.

In [0]:
fifa_df.select(["nationality"]).distinct().show(n=10, truncate=False)

In [0]:
fifa_df.select(["club", "nationality"]).distinct().show(n=10, truncate=False)

### **2.2 Find Duplicates (if any)**

In [0]:
print("The total number of duplicated ages are {:d} out of {:d}".
      format(fifa_df.count() - fifa_df.dropDuplicates(["age"]).count(), fifa_df.count()))

### **2.3 Filtering Data**
In order to filter data, according to the condition specified, we use the <code>filter</code> command. Here we are filtering our DataFrame based on the condition that <code>team_jersey_number</code> must be equal to <code>10</code> and then we are calculating how many records/rows are there in the filtered output.

In [0]:
fifa_df.filter(fifa_df.team_jersey_number==10).show(truncate=True)

In [0]:
print("The total number of players having jersey #10 are: {:d}".
      format(fifa_df.filter(fifa_df.team_jersey_number==10).count()))

### **2.4 Filtering Data (using multiple parameters)**
We can filter our data based on multiple logical conditions connected together by <code>AND</code> or <code>OR</code> operators.

In [0]:
# Filter Italian goalkeepers only
fifa_df.filter((fifa_df.team_position=="GK") & (fifa_df.nationality=="Italy")).show()

### **2.5 Sorting Data**

We can sort data using the <code>sort</code> method _or_ the <code>orderBy</code> method (which is just an alias of the former).

In [0]:
fifa_df.sort(["height_cm", "weight_kg"], ascending=[False, False]).show()
# Alternatively:
# fifa_df.orderBy(["height_cm", "weight_kg"], ascending=[False, False]).show()

### **2.6 Filtering + Sorting Data**

In [0]:
# Filter Italian goalkeepers only, and sort them by height in ascending order
fifa_df.filter((fifa_df.team_position=="GK") & (fifa_df.nationality=="Italy")).sort("height_cm").show()

### **2.7 Grouping Data**

In order to group data in a <code>DataFrame</code> on the basis of the values of one or more column, the <code>groupBy</code> operator is used.

In [0]:
# Let's group data by `age`
fifa_df.groupby(["age"]).count().show(10)

In [0]:
# Let's group data by `age` and sort it according to the resulting count in non-ascending order
fifa_df.groupby(["age"]).count().sort("count", ascending=False).show(10)

In [0]:
# Let's group data by `nationality` and `team_position`
fifa_df.groupby(["nationality", "team_position"]).count().sort("count", ascending=False).show(10)

In [0]:
# Let's group data by `nationality` and `team_position` (except "SUB" and "RES")
fifa_df.filter((fifa_df.team_position != "SUB") & (fifa_df.team_position != "RES")).groupby(["nationality", "team_position"]).count().sort("count", ascending=False).show(10)

### **2.8 Performing SQL Queries**

We can also pass SQL queries directly to any DataFrame, for that we need to create a table from the DataFrame using the <code>registerTempTable</code> method and then use  <code>sqlContext.sql()</code> to pass the SQL queries.

In [0]:
fifa_df.registerTempTable('fifa_table')
spark.sql('select * from fifa_table').show(10)

In [0]:
spark.sql('select distinct(club) from fifa_table').show(10)

## **3. Data Visualization**

A crucial step of the typical _exploratory analysis_ phase is to visually inspect data in order to get a sense of possible insights (e.g., _outliers_, _relationships_, _patterns_).

PySpark **does not** have any plotting functionality (yet). If we want to plot something, we **must** bring the data out of the Spark (_distributed_) Context and into our _local_ Python session, where we can deal with it using any of Python's many plotting libraries, such as <code>matplotlib</code> and <code>seaborn</code>. Please, refer to _Lecture 10_ available at this [link](https://github.com/gtolomei/python-for-datascience#Class-Schedule) for a deeper understanding of these libraries.

**NOTE:** Databricks provides itself support for visualization through the ```display``` function, although ```matplotlib``` and ```seaborn``` may guarantee more flexibility (see [here](https://docs.databricks.com/notebooks/visualizations/index.html) for more information).

In [0]:
# The following import allows to use Python's plotting APIs: matplotlib and seaborn
import matplotlib.pyplot as plt
import seaborn as sns
# The following directive is to allow the inline visualization of generated plots
#%matplotlib inline 

### **3.1 Move Data out from <code>SparkContext</code> into <code>Pandas</code>**

This can be achieved by calling the method <code>toPandas</code> on our original Spark DataFrame.

In [0]:
# Convert our `fifa_df` Spark DataFrame into the corresponding Pandas DataFrame
p_fifa_df = fifa_df.toPandas()

### **3.2 Plot <code>age</code> Distribution using <code>Pandas</code>' built-in plotting functions**

In [0]:
# Create a 1x1 figure
fig, ax = plt.subplots(1,1, figsize=(8,6))

_ = p_fifa_df.age.plot.hist(ax=ax, bins=10, color="lightblue", edgecolor='black', linewidth=1.2)
_ = p_fifa_df.age.plot.density(ax=ax, secondary_y=True, color="red")
_ = ax.set_xlabel("Age")
_ = ax.set_ylabel("Frequency")

### **3.3 Plot <code>age</code> Distribution using <code>matplotlib</code>'s plotting functions**

In [0]:
# Create a 1x1 figure
fig, ax = plt.subplots(1,1, figsize=(8,6))

_ = ax.hist(p_fifa_df.age, bins=10, color="lightblue", edgecolor='black', linewidth=1.2)
_ = ax
_ = ax.set_xlabel("Age")
_ = ax.set_ylabel("Frequency")

### **3.4 Plot <code>age</code> Distribution using <code>seaborn</code>'s plotting functions**

In [0]:
# Create a 1x1 figure
fig, ax = plt.subplots(1,1, figsize=(8,6))

_ = sns.distplot(p_fifa_df.age, 
                 bins=10, 
                 kde=True,
                 ax=ax, 
                 color="lightblue", 
                 hist_kws={"edgecolor":"k", "linewidth":1.2},
                 kde_kws={"color": "r", "lw": 1.5}
                 )
_ = ax.set_xlabel("Age")
_ = ax.set_ylabel("Density(%)")

### **3.5 Boxplot of a single variable**

In [0]:
# Create a 1x1 figure
fig, ax = plt.subplots(1,1, figsize=(6,4))

_ = sns.boxplot(x=p_fifa_df.height_cm, color="g")
# Alternatively:
#_ = sns.boxplot(x="height_cm", data=p_fifa_df, color="g")
_ = ax.set_xlabel("Height (cm)")

### **3.6 Boxplot combining 2 variables**

In [0]:
# Create a 1x1 figure
fig, ax = plt.subplots(1,1, figsize=(8,6))

_ = sns.boxplot(x="team_position", y="height_cm", data=p_fifa_df)
_ = ax.set_xlabel("Team Position")
_ = ax.set_ylabel("Height (cm)")

In [0]:
# Let's restrict the same plot above to the top-k position, escept for "SUB" and "RES"
k = 10
# Filter out `SUB` and `RES` players
filtered_df = p_fifa_df[(p_fifa_df.team_position != "SUB") & (p_fifa_df.team_position != "RES")]
# Find out what are the top-k roles
top_k_roles = filtered_df["team_position"].value_counts()[:k].index.values
print("The top-{:d} roles are: {:s}".format(k, ", ".join(top_k_roles)))
# Select only those top-k players
filtered_df = filtered_df[(filtered_df.team_position.isin(top_k_roles))]

In [0]:
# Create a 1x1 figure
fig, ax = plt.subplots(1,1, figsize=(8,6))

_ = sns.boxplot(x="team_position", y="height_cm", data=filtered_df)
_ = ax.set_xlabel("Team Position")
_ = ax.set_ylabel("Height (cm)")

### **3.7 Pairplot: Showing relationship between 2 or more variables**

In [0]:
_ = sns.pairplot(p_fifa_df[["height_cm", "weight_kg"]],
                 kind="reg",
                 diag_kind="hist", # hist or kde
                 diag_kws={"color":"r", "alpha":0.7, "edgecolor":"k"},     
                 height=3
                 )